<h2>This notebook studies the <b>Team Orienteering </b> with Pick-up, Delivery Problem and Transfer (TOPDPT)</h2>

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os, sys, random, time, pickle
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

from gurobipy import Model, quicksum, GRB
import numpy as np
from util import generate_node_cargo_size_change, read_pickle, group_cycle_truck, manual_stop
from pathlib import Path
from tvopdpt import tvopdpt_milp_gurobi
dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
num_ins = 10

In [3]:
def solve_tvopdpt_mip(ins,  # dict contains the data of pdpt instance,
                     gurobi_log_file, # file where all data of pdotw solutions are saved
                     optimize_pdotw_routes = True,
                     max_runtime = 100,
                     verbose = 0):  

    # load data from ins
    selected_truck = ins['truck']
    selected_cargo = ins['cargo']
    selected_node = ins['nodes']
    selected_edge = ins['edge_shortest']    
    # edges = ins['edges']
    # nodes = ins['nodes']
    constant = ins['constant']
    node_cargo_size_change = ins['node_cargo_size_change']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = ins['single_truck_deviation']
    
    print(f'========= START TVOPDPT =========')
    print(f'   +++ with [{len(selected_truck.keys())}] Truck {list(selected_truck.keys())}  ')

    print(f'   +++ with [{len(selected_cargo.keys())}] cargo {list(selected_cargo.keys())}  ')
    print(f'   +++ with [{len(selected_node)}] cargo {selected_node}  ')

    

    # edge_shortest, path_shortest = replace_edge_by_shortest_length_nx(nodes, edges)
    # single_truck_deviation = calculate_single_truck_deviation(truck, cargo, edge_shortest)
    
    start_time = time.time()

    
    created_truck = selected_truck.copy()
    
    # nodes in the cluster
    # Note. cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time','departure_node', 'arrival_node']
    # truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']
    

    if verbose >0:
        print(f'+++ Preprocess data to instantiate a PDOTW MIP')
    if verbose > 2:
        print(f'    [selected_cargo] size: {len(selected_cargo)}')
        for key, value in selected_cargo.items():
            print(f'        {key, value}')
        print(f'    [created_truck] size: {len(created_truck)}')
        for key, value in created_truck.items():
            print(f'       {key, value}')
    
    ### Need to update node_cargo_size_change 
    node_cargo_size_change = generate_node_cargo_size_change(selected_node, selected_cargo)

    ### group cycle and non-cycle trucks
    created_truck_yCycle, created_truck_nCycle, selected_truck = group_cycle_truck(created_truck)  
    
    if verbose > 2:
        print('    [created_truck_yCycle]', created_truck_yCycle)
        print('    [created_truck_nCycle]', created_truck_nCycle)


    if verbose >0:
        print(f'+++ Solve TVOPDPT MILP in Gurobi')
    ### use gurobi to solve the GROW origin PDPTW
    # Note. the pdotw_mip_gurobi function is desgined to take the same arguments as pdpt function
    # but some parameters

    obj_val_MP, runtime_MP, gurobi_res = tvopdpt_milp_gurobi(constant, 
                                                           selected_cargo, 
                                                           selected_truck, 
                                                           created_truck_yCycle, 
                                                           created_truck_nCycle,
                                                           selected_node, 
                                                           selected_edge, 
                                                           node_cargo_size_change, 
                                                           max_runtime, 
                                                           gurobi_log_file, 
                                                           verbose = 1)

    x_sol, z_sol, u_sol, w_sol, S_sol, D_sol, A_sol, Sb_sol, Db_sol, Ab_sol = gurobi_res
    # Index k for truck is pre-defined
    #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
    #z_sol: z^{kr}_{ij}, if truck k visit edge (i,j) with cargo r
    #u_sol: u^r_i, if cargo r is transfered at node i
    #y_sol: y^k_r, if parcel r is carried by truck k
    #S_sol: x^k_i, total size of cargos on truck k at node i
    #D_sol: D^k_i, depature time of truck k at node i
    #A_sol: A^k_i, arrival time of truck k at node i

    print(f'=== Result: total cargo [{len(selected_cargo.keys())}], MIP obj [{obj_val_MP}]')
    res = {'obj_val_MP': obj_val_MP,
           'runtime_MP': runtime_MP,
           'x_sol':  x_sol,
           'z_sol':  z_sol,
           'u_sol':  u_sol,
           'w_sol':  w_sol,
           'S_sol':  S_sol,
           'D_sol':  D_sol,
           'A_sol':  A_sol,
           'Sb_sol': Sb_sol,
           'Db_sol': Db_sol,
           'Ab_sol': Ab_sol
          }

    return res

In [4]:
def test_subproblem():
    max_runtime = 100
    case_num, num_trucks = 1, 2

    for ins_idx in range(1):
        pdpt_ins_filename = os.path.join(dir_, 'out', 'subproblem.pkl')
        print(f'===== START team orienteering pdpt\n      {pdpt_ins_filename}')
        pdpt_ins = read_pickle(pdpt_ins_filename)

        path_ = os.path.join(dir_, f'data/case{case_num}', 'tvopdpt_res')
        Path(path_).mkdir(parents=True, exist_ok=True)

        gurobi_log_file = os.path.join(dir_, 'out', 'subproblem_gurobi.pkl')

        res = solve_tvopdpt_mip(pdpt_ins,  # dict contains the data of pdpt instance,
                            gurobi_log_file, # file where all data of pdotw solutions are saved
                            optimize_pdotw_routes = True,
                            max_runtime = max_runtime,
                            verbose = 0)

        res_filename = os.path.join(dir_, 'out', 'subproblem_res.pkl')
        with open(res_filename, 'wb') as f:
            pickle.dump(res, f)
# test_subproblem()

In [5]:
def main():
    max_runtime = 100
    case_num, num_trucks = 1, 2

    for ins_idx in range(1):
        pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
        print(f'===== START team orienteering pdpt\n      {pdpt_ins_filename}')
        pdpt_ins = read_pickle(pdpt_ins_filename)

        path_ = os.path.join(dir_, f'data/case{case_num}', 'tvopdpt_res')
        Path(path_).mkdir(parents=True, exist_ok=True)

        gurobi_log_file = os.path.join(path_, f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_gurobi.pkl')

        res = solve_tvopdpt_mip(pdpt_ins,  # dict contains the data of pdpt instance,
                            gurobi_log_file, # file where all data of pdotw solutions are saved
                            optimize_pdotw_routes = True,
                            max_runtime = max_runtime,
                            verbose = 0)

        res_filename = os.path.join(path_, f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_res.pkl')
        with open(res_filename, 'wb') as f:
            pickle.dump(res, f)

# main()

In [6]:
def check_mip_sol():
    case_num, ins_idx, num_trucks = 1, 0, 2
    #pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
    pdpt_ins_filename = os.path.join(dir_, 'out', 'subproblem.pkl')

    print(f'===== START team orienteering pdpt\n      {pdpt_ins_filename}')
    pdpt_ins = read_pickle(pdpt_ins_filename)

    # load data from ins
    selected_truck = pdpt_ins['truck']
    selected_cargo = pdpt_ins['cargo']
    selected_node = pdpt_ins['nodes']
    print(selected_node)
    selected_edge = pdpt_ins['edge_shortest']    
    # edges = ins['edges']
    # nodes = ins['nodes']
    constant = pdpt_ins['constant']
    node_cargo_size_change = pdpt_ins['node_cargo_size_change']
    edge_shortest = pdpt_ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = pdpt_ins['single_truck_deviation']

    print(f'total num. of cargo {len(selected_cargo.keys())}')

    #tvopdpt_res_filename = os.path.join(dir_, f'data/case{case_num}/tvopdpt_res', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_res.pkl')
    tvopdpt_res_filename = os.path.join(dir_, 'out', 'subproblem_res.pkl')
    tvopdpt_res = read_pickle(tvopdpt_res_filename)

    print(tvopdpt_res.keys())

    print('obj', tvopdpt_res['obj_val_MP'])
    print('runtime_MP', tvopdpt_res['runtime_MP'])
    z_sol = tvopdpt_res['z_sol']
    u_sol = tvopdpt_res['u_sol']
    w_sol = tvopdpt_res['w_sol']
    x_sol = tvopdpt_res['x_sol']
    s_sol = tvopdpt_res['S_sol']

    print('=== Check transfer')
    truck_1, truck_2 = selected_truck.keys()
    for cargo_key, cargo_value in selected_cargo.items():
        cargo_origin, cargo_destination = cargo_value[-2], cargo_value[-1]
        for node_ in selected_node:
            if w_sol[(node_, cargo_key)] == 1:
                print(f' +++ {cargo_key} is transferred from Truck [{truck_2}] to Truck [{truck_1}] at node [{node_}]')
            if u_sol[(node_, cargo_key)] == 1:
                print(f' +++ {cargo_key} is transferred from Truck [{truck_1}] to Truck [{truck_2}] at node [{node_}]')

    print('=== truck route')

    for truck_key in selected_truck:
        truck_origin, truck_des = selected_truck[truck_key][:2] # starting from the origin node of each truck
        print(f' +++ [{truck_key}], origin [{truck_origin}], dest [{truck_des}]')
        node_curr = truck_origin
        for node_ in selected_node:
            if node_ != node_curr and x_sol[(node_curr, node_, truck_key)] == 1: # find node_ such that x[source, node_, truck_key] == 1
                print(f'  ++++++ {truck_key} from origin [{node_curr}] to node [{node_}]')
#                 print(f'  ++++++ s_sol[({node_}, {truck_key})]: [{s_sol[(node_, truck_key)]}]')
                node_curr = node_
                break
        while node_curr != selected_truck[truck_key][1]: # terminate when reach the arival node of each truck
            for node_ in selected_node:
                if node_ != node_curr and x_sol[(node_curr, node_, truck_key)] == 1: # find node_ such that x[source, node_, truck_key] == 1
                    if node_ == truck_des:
                        print(f'  ++++++ {truck_key} from node [{node_curr}] to dest [{node_}]')
                    else:
                        print(f'  ++++++ {truck_key} from node [{node_curr}] to node [{node_}]')
                            
                    node_curr = node_
                    break
                    
#     for node_curr in selected_node:
#         for truck_key in selected_truck.keys():
#             if node_curr !=  selected_truck[truck_key][1]:
#                 print(f's_sol[({node_curr}, {truck_key})] {s_sol[(node_curr, truck_key)]}')
#                 print(f'sum_z * size {sum([z_sol[(node_curr, node_next, truck_key, cargo_key)]* selected_cargo[cargo_key][0] for node_next in selected_node for cargo_key in selected_cargo.keys() if node_next != node_curr]) }')
    
    print('=== cargo route')

    # truck1, truck2 = selected_truck.keys()
    cargo_delivered = []
    cargo_undelivered = []
    for cargo_key, cargo_value in selected_cargo.items():
        cargo_origin, cargo_dest = cargo_value[-2:]
        print(f'   +++ [{cargo_key}], origin [{cargo_origin}], dest [{cargo_dest}]')
        if sum([sum([z_sol[(node, cargo_dest, truck_key, cargo_key)]for node in selected_node if node != cargo_dest]) for truck_key in selected_truck.keys()]) == 1:
            cargo_delivered.append(cargo_key)
            node_curr = cargo_origin
            for node_next in selected_node:
                z_temp = 0
                if node_next!=node_curr:
                    for truck_key in selected_truck.keys():
                        z_temp += z_sol[(cargo_origin, node_next, truck_key, cargo_key)]
                        if z_sol[(cargo_origin, node_next, truck_key, cargo_key)] == 1:
                            print(f'  ++++++ {cargo_key} from origin [{node_curr}] to node [{node_next}] by Truck {truck_key}')
                            print(f'  ++++++ z_sol[({cargo_origin}, {node_next}, {truck_key}, {cargo_key})] = {z_sol[(cargo_origin, node_next, truck_key, cargo_key)]}' )
                            node_curr = node_next
                            
                        if z_temp == 1:
                            break
                if z_temp == 1:
                    break
            while node_curr != cargo_dest:
                z_temp = 0
                for node_next in selected_node:
                    if node_next!=node_curr:
                        for truck_key in selected_truck.keys():
                            z_temp += z_sol[(node_curr, node_next, truck_key, cargo_key)]
                            if z_sol[(node_curr, node_next, truck_key, cargo_key)] == 1:
                                if node_next == cargo_dest:
                                    print(f'  ++++++ {cargo_key} from node [{node_curr}] to dest [{node_next}] by Truck {truck_key}')
                                else:
                                    print(f'  ++++++ {cargo_key} from node [{node_curr}] to node [{node_next}] by Truck {truck_key}')
                                print(f'  ++++++ z_sol[({node_curr}, {node_next}, {truck_key}, {cargo_key})] = {z_sol[(node_curr, node_next, truck_key, cargo_key)]}')

                                node_curr = node_next

                                
                            
                            if z_temp == 1:
                                break
                    if z_temp == 1:
                        break
        else:
            cargo_undelivered.append(cargo_key)

            # print(f'[{cargo_key}] is not delivered')

    for cargo_key in selected_cargo.keys():
        for node in selected_node:
            if w_sol[(node, cargo_key)] == 1:
                print(f'w_sol[({node}, {cargo_key})] {w_sol[(node, cargo_key)]}')
                for node_2 in selected_node:
                    for truck_key in selected_truck.keys():
                        if node_2 != node:
                            if z_sol[(node, node_2, truck_key, cargo_key)] == 1:
                                print(f'   z_sol[({node}, {node_2}, {truck_key}, {cargo_key})], {z_sol[(node, node_2, truck_key, cargo_key)]}')
                            if z_sol[(node_2, node, truck_key, cargo_key)] == 1:
                                print(f'   z_sol[({node_2}, {node}, {truck_key}, {cargo_key})], {z_sol[(node_2, node, truck_key, cargo_key)]}')
            

    print(f'{len(cargo_delivered)} cargo delivered: {cargo_delivered}')
    print(f'{len(cargo_undelivered)} cargo undelivered: {cargo_undelivered}')


    

# check_mip_sol()

In [9]:
def postprocess_tvopdpt_solution(tvopdpt_ins, tvopdpt_res, verbose = 0):

    selected_cargo = tvopdpt_ins['cargo']
    selected_truck = tvopdpt_ins['truck']
    assert len(selected_truck.keys()) == 2
    truck1, truck2 = selected_truck.keys()
    selected_node  = tvopdpt_ins['nodes']
    selected_edge  = tvopdpt_ins['edge_shortest']
    
    x_sol = tvopdpt_res['x_sol']
    z_sol = tvopdpt_res['z_sol']
    z_sol = tvopdpt_res['z_sol']
    u_sol = tvopdpt_res['u_sol']
    w_sol = tvopdpt_res['w_sol']

    truck_used = []         # list of trucks used in the solution
    
    # dictionary, for each truck_key, there is a list of cargo carried that was on this truck. E.g., {'T1": ['C1', 'C2', ...]}
    cargo_in_truck = {}   
    for truck_key in selected_truck.keys():
        cargo_in_truck[truck_key] = []

    # dictionary, for each cargo_key, there is a list of truck that carried this cargo. E.g., {'C1": ['T1', 'T2', ...]}
    trucks_per_cargo = {} 
    for cargo_key in selected_cargo.keys():
        trucks_per_cargo[cargo_key] = []

    cargo_delivered = []   # list of delivered cargo
    cargo_undelivered = [] # list of undelivered cargo


    truck_route = {}
    for truck_key in selected_truck.keys():
        truck_route[truck_key] = []
    cargo_route = {}
    for cargo_key in selected_cargo.keys():
        cargo_route[cargo_key] = []

    # dict, for each cargo_key, there is a nested list, each with [transfer_node, truck_from, truck_to]
    cargo_transfer = {}
    for cargo_key in selected_cargo.keys():
        cargo_transfer[cargo_key] = []

    if verbose >0:
        print(f'   +++ Process cargo-to-truck-assignment')
    # Generate cargo_in_truck
    for cargo_key, cargo_value in selected_cargo.items():
        cargo_origin, cargo_dest = cargo_value[-2:]
        # assert sum([z_sol[(cargo_origin, node_, truck1, cargo_key)] for node_ in selected_node if node_!= cargo_origin]) + sum([z_sol[(cargo_origin, node_, truck2, cargo_key)] for node_ in selected_node if node_!= cargo_origin]) == 1
        # assert sum([z_sol[(node_, cargo_dest, truck1, cargo_key)] for node_ in selected_node if node_!= cargo_dest]) +sum([z_sol[(cargo_dest, node_, truck2, cargo_key)] for node_ in selected_node if node_!= cargo_dest]) == 1

        for truck_key in selected_truck.keys():
            if sum([z_sol[(cargo_origin, node_, truck_key, cargo_key)] for node_ in selected_node if node_!= cargo_origin]) + sum([z_sol[(node_, cargo_dest, truck_key, cargo_key)] for node_ in selected_node if node_!= cargo_dest])  >=1:
                cargo_in_truck[truck_key].append(cargo_key)


    if verbose >0:
        print(f'   +++ Process truck_route')
    # postprocess truck_route

    for truck_key in selected_truck:
        truck_origin, truck_des = selected_truck[truck_key][:2] # starting from the origin node of each truck
        print(f'[{truck_key}], origin [{truck_origin}], dest [{truck_des}]')
        node_curr = truck_origin
        for node_ in selected_node:
            if node_ != node_curr and x_sol[(node_curr, node_, truck_key)] == 1: # find node_ such that x[source, node_, truck_key] == 1
                print(f'  +++ {truck_key} from origin [{node_curr}] to node [{node_}]')
                if len(truck_route[truck_key]) == 0: # append source as the first node
                        truck_route[truck_key].append(truck_origin)
                truck_route[truck_key].append(node_)

                node_curr = node_
                break
        while node_curr != selected_truck[truck_key][1]: # terminate when reach the arival node of each truck
            for node_ in selected_node:
                if node_ != node_curr and x_sol[(node_curr, node_, truck_key)] == 1: # find node_ such that x[source, node_, truck_key] == 1
                    if node_ == truck_des:
                        print(f'  +++ {truck_key} from node [{node_curr}] to dest [{node_}]')
                    else:
                        print(f'  +++ {truck_key} from node [{node_curr}] to node [{node_}]')
                    truck_route[truck_key].append(node_)

                    node_curr = node_
                    break
    # RECALL, cargo is a dictionary with the following format:
    # cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time', 'departure_node', 'arrival_node']

    if verbose >0:
        print(f'   +++ Process cargo_route, cargo_delivered and cargo_undelivered')
    # truck1, truck2 = selected_truck.keys()
    cargo_delivered = []
    cargo_undelivered = []
    for cargo_key, cargo_value in selected_cargo.items():
        cargo_origin, cargo_dest = cargo_value[-2:]
        print(f'[{cargo_key}], origin [{cargo_origin}], dest [{cargo_dest}]')
        if sum([sum([z_sol[(node, cargo_dest, truck_key, cargo_key)]for node in selected_node if node != cargo_dest]) for truck_key in selected_truck.keys()]) == 1:
            cargo_delivered.append(cargo_key)
            node_curr = cargo_origin
            for node_next in selected_node:
                z_temp = 0
                if node_next!=node_curr:
                    for truck_key in selected_truck.keys():
                        z_temp += z_sol[(cargo_origin, node_next, truck_key, cargo_key)]
                        if z_sol[(cargo_origin, node_next, truck_key, cargo_key)] == 1:
                            print(f'  +++ {cargo_key} from origin [{node_curr}] to node [{node_next}] by Truck {truck_key}')
                            if len(cargo_route[cargo_key]) == 0: # append source as the first node
                                cargo_route[cargo_key].append((truck_key, node_curr))
                            cargo_route[cargo_key].append((truck_key, node_next))

                            node_curr = node_next
                        if z_temp == 1:
                            break
                if z_temp == 1:
                    break
                    
            while node_curr != cargo_dest:
                z_temp = 0
                for node_next in selected_node:
                    if node_next!=node_curr:
                        for truck_key in selected_truck.keys():
                            z_temp += z_sol[(node_curr, node_next, truck_key, cargo_key)]
                            if z_sol[(node_curr, node_next, truck_key, cargo_key)] == 1:
                                if node_next == cargo_dest:
                                    print(f'  +++ {cargo_key} from node [{node_curr}] to dest [{node_next}] by Truck {truck_key}')
                                    
                                else:
                                    print(f'  +++ {cargo_key} from node [{node_curr}] to node [{node_next}] by Truck {truck_key}')
                                cargo_route[cargo_key].append((truck_key, node_next))

                                node_curr = node_next                            
                            if z_temp == 1:
                                break
                    if z_temp == 1:
                        break
        else:
            cargo_undelivered.append(cargo_key)

            print(f'[{cargo_key}] is not delivered')

    # for cargo_key, cargo_value in selected_cargo.items():
    #     cargo_origin, cargo_dest = cargo_value[-2:]
    #     # if cargo is delivered
    #     if sum([z_sol[(node_, cargo_dest, truck_key, cargo_key)] for node_ in selected_node for truck_key in selected_truck.keys() if node_!= cargo_dest]) == 1:
    #         cargo_delivered.append(cargo_key)
    #         print(f'cargo [{cargo_key}] is delivered')

    #         #start from cargo_origin
    #         node_curr = cargo_origin
    #         #check the next node
    #         for n_next in selected_node:
    #             z_temp = 0
    #             if n_next != node_curr:
    #                 for truck_key in selected_truck.keys():
    #                     z_temp += z_sol[(node_curr, n_next, truck_key, cargo_key)]
    #                     if z_sol[(node_curr, n_next, truck_key, cargo_key)] == 1:
    #                         if len(cargo_route[cargo_key]) == 0: # append source as the first node
    #                             cargo_route[cargo_key].append((truck_key, node_curr))
    #                         cargo_route[cargo_key].append((truck_key, n_next))
    #                         node_curr = n_next
    #                         print(f'cargo_origin [{cargo_origin}], node_next[{n_next}]')
    #                         break

    #         while node_curr != cargo_value[-1]:
    #             for n_next in selected_node:
    #                 if n_next!=node_curr:
    #                     for truck_key in selected_truck.keys():
    #                         if  z_sol[(node_curr, n_next, truck_key, cargo_key)] == 1:
    #                             cargo_route[cargo_key].append((truck_key, n_next))
    #                             node_curr = n_next
    #                             break
 
    #     else:
    #         print([ [truck_key, cargo_key, sum(z_sol[(node_, cargo_dest, truck_key, cargo_key)] for node_ in selected_node if node_!= cargo_dest) ] for truck_key in selected_truck.keys()])

    #         assert sum([z_sol[(node_, cargo_dest, truck_key, cargo_key)] for node_ in selected_node for truck_key in selected_truck.keys() if node_!= cargo_dest]) == 0
    #         print(f'cargo [{cargo_key}] is undelivered')

    #         cargo_undelivered.append(cargo_key)

    truck_1, truck_2 = selected_truck.keys()
    if verbose >0:
        print(f'   +++ Process cargo_transfer')
    for cargo_ in selected_cargo.keys():
        for node_curr in selected_node:
            if u_sol[node_, cargo_key] == 1:
                assert sum([z_sol[(node_prev, node_curr, truck_1, cargo_)] for node_prev in selected_node if node_curr != node_prev]) - sum([z_sol[(node_curr, node_next, truck_2, cargo_)] for node_next in selected_node if node_curr != node_next]) == 1
                cargo_transfer[cargo_key].append([node_curr, truck_1, truck_2])
            if w_sol[node_curr, cargo_key] == 1:
                assert sum([z_sol[(node_prev, node_curr, truck_2, cargo_)] for node_prev in selected_node]) - sum([z_sol[(node_curr, node_next, truck_1, cargo_)] for node_next in selected_node]) == 1
                cargo_transfer[cargo_key].append([node_curr, truck_2, truck_1])
                



    processed_res = (truck_used, cargo_delivered, cargo_undelivered, cargo_in_truck, truck_route, cargo_route, cargo_transfer)


    return processed_res


def compute_cost(pdpt_ins, 
                 truck_route, 
                 truck_use, 
                 cargo_transfer,
                 verbose = 0):
                 
    truck_cost, travel_cost, transfer_cost = 0, 0, 0

    constant = pdpt_ins['constant']
    edge_list = pdpt_ins['edge_shortest']
    cargo_list = pdpt_ins['cargo']

    ###### truck_cost ######

    for truck_key in truck_use:
        assert truck_key in truck_route.keys()
        truck_cost += constant['truck_fixed_cost']


    ###### travel_cost ######
    for truck_key in truck_use:
        route_ = truck_route[truck_key]
        route_cost = 0
        for i in range(len(route_)-1):
            route_cost += constant['truck_running_cost'] * edge_list[(route_[i], route_[i+1])]
        travel_cost+=route_cost

    ###### transfer_cost ######
    for cargo_key in cargo_transfer.keys():
        for transfer in cargo_transfer[cargo_key]:
            transfer_cost += int(cargo_list[cargo_key][0] * constant['cargo_reloading_cost'])

    if verbose > 0:
        print('\nThe truck_cost:', truck_cost)
        print('The travel_cost:', travel_cost)
        print('The transfer_cost:', transfer_cost, '\n')

    return (truck_cost, travel_cost, transfer_cost)

def read_result(case_num):
    pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}.pkl')
    pdotw_res_filename = os.path.join(dir_, 'out', 'iniSol', f'case{case_num}_iniSol.pkl')

    tvopdpt_ins_file = os.path.join(dir_, 'out', 'impSol_tvopdpt', f'case{case_num}_sp1.pkl')
    tvopdpt_res_file = os.path.join(dir_, 'out', 'impSol_tvopdpt', f'case{case_num}_sp1_res.pkl')

    pdpt_ins = read_pickle(pdpt_ins_filename)

    pdotw_sol = read_pickle(pdotw_res_filename)

    pdpt_truck_used = pdotw_sol['route']['used_truck']
    pdpt_truck_route = pdotw_sol['route']['truck_route']
    pdpt_cargo_route = pdotw_sol['route']['cargo_route']
    pdpt_cargo_transfer = {}

    pdotw_cost = compute_cost(pdpt_ins, 
                                pdpt_truck_route, 
                                pdpt_truck_used, 
                                pdpt_cargo_transfer)
                        

    pdpt_truck_used.remove('T1') # read from log file, todo update this after destory solution
    tvopdpt_ins = read_pickle(tvopdpt_ins_file)

    tvopdpt_sol = read_pickle(tvopdpt_res_file)

    tvopdpt_res = postprocess_tvopdpt_solution(tvopdpt_ins, tvopdpt_sol, verbose = 1)  

    tvopdpt_truck_used = tvopdpt_res[0]
    tvopdpt_truck_route = tvopdpt_res[-3]
    tvopdpt_cargo_route = tvopdpt_res[-2]
    tvopdpt_cargo_transfer= tvopdpt_res[-1]

    print(' ==== update PDPT results from tvopdpt results')
    for truck_key in tvopdpt_truck_used:
        assert truck_key in pdpt_truck_route.keys()
        assert truck_key in pdpt_truck_used
        print(f'truck route for [{truck_key}] in pdotw {pdpt_truck_route[truck_key]}')
        pdpt_truck_route[truck_key] = tvopdpt_truck_route[truck_key]  # update truck route with solution from tvopdpt
        print(f'truck route for [{truck_key}] in tvopdpt {pdpt_truck_route[truck_key]}')

    for cargo_key in tvopdpt_cargo_route.keys():
        assert cargo_key in pdpt_cargo_route.keys()
        print(f'truck route for [{cargo_key}] in pdotw {pdpt_cargo_route[cargo_key]}')
        pdpt_cargo_route[cargo_key] = tvopdpt_cargo_route[cargo_key]  # update cargo route with solution from tvopdpt
        print(f'truck route for [{cargo_key}] in tvopdpt {pdpt_cargo_route[cargo_key]}')

    for cargo_key in tvopdpt_cargo_transfer.keys():
        if len(tvopdpt_cargo_transfer[cargo_key]) >0:
            print(f'transfer cargo [{cargo_key}]')
            pdpt_cargo_transfer[cargo_key] = tvopdpt_cargo_transfer[cargo_key]
        else:
            print(f'no transfer for cargo [{cargo_key}]')

            



    pdpt_cost = compute_cost(pdpt_ins, 
                            pdpt_truck_route, 
                            pdpt_truck_used, 
                            pdpt_cargo_transfer)

    
    print(f'++ Init_sol: truck_cost: [{pdotw_cost[-3]}], travel_cost: [{pdotw_cost[-2]}], transfer_cost: [{pdotw_cost[-1]}]')

    print(f'++ Impro_sol: truck_cost: [{pdpt_cost[-3]}], travel_cost: [{pdpt_cost[-2]}], transfer_cost: [{pdpt_cost[-1]}]')

    
read_result(3)


The truck_cost: 450000.0
The travel_cost: 442900.0
The transfer_cost: 0 

   +++ Process cargo-to-truck-assignment
   +++ Process truck_route
[T7], origin [N21], dest [N4]
  +++ T7 from origin [N21] to node [N7]
  +++ T7 from node [N7] to node [N6]
  +++ T7 from node [N6] to dest [N4]
[T16], origin [N24], dest [N25]
  +++ T16 from origin [N24] to node [N6]
  +++ T16 from node [N6] to dest [N25]
   +++ Process cargo_route, cargo_delivered and cargo_undelivered
[C16], origin [N24], dest [N25]
  +++ C16 from origin [N24] to node [N6] by Truck T16
  +++ C16 from node [N6] to dest [N25] by Truck T16
[C26], origin [N21], dest [N7]
  +++ C26 from origin [N21] to node [N7] by Truck T7
[C95], origin [N24], dest [N6]
  +++ C95 from origin [N24] to node [N6] by Truck T16
   +++ Process cargo_transfer
 ==== update PDPT results from tvopdpt results
truck route for [C16] in pdotw [('T1', 'N24'), ('T1', 'N25')]
truck route for [C16] in tvopdpt [('T16', 'N24'), ('T16', 'N6'), ('T16', 'N25')]
truck ro

In [8]:
def read_pdotw_cost():
    for case_num in range(1,6,1):
    # for case_num in [3,]:
        print(f'++++ case {case_num}')
        pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}.pkl')

        pdotw_res_filename = os.path.join(dir_, 'out', 'iniSol', f'case{case_num}_iniSol.pkl')

        pdpt_ins = read_pickle(pdpt_ins_filename)
        pdotw_sol = read_pickle(pdotw_res_filename)

        pdotw_truck_used = pdotw_sol['route']['used_truck']
        pdotw_truck_route = pdotw_sol['route']['truck_route']
        pdotw_cargo_route = pdotw_sol['route']['cargo_route']


        pdotw_cost = compute_cost(pdpt_ins, 
                                pdotw_truck_route, 
                                pdotw_truck_used, 
                                {})

    print(f'++ Init_sol: [{len(pdotw_truck_used)}] used, truck_cost: [{pdotw_cost[-3]}], travel_cost: [{pdotw_cost[-2]}], transfer_cost: [{pdotw_cost[-1]}]')



read_pdotw_cost()

++++ case 1

The truck_cost: 570000.0
The travel_cost: 488600.0
The transfer_cost: 0 

++++ case 2

The truck_cost: 450000.0
The travel_cost: 412200.0
The transfer_cost: 0 

++++ case 3

The truck_cost: 450000.0
The travel_cost: 442900.0
The transfer_cost: 0 

++++ case 4

The truck_cost: 480000.0
The travel_cost: 417900.0
The transfer_cost: 0 

++++ case 5

The truck_cost: 600000.0
The travel_cost: 505900.0
The transfer_cost: 0 

++ Init_sol: truck_cost: [600000.0], travel_cost: [505900.0], transfer_cost: [0]
